In [1]:
import requests
from zipfile import ZipFile
from io import BytesIO

url = 'https://raw.githubusercontent.com/OTRF/Security-Datasets/master/datasets/atomic/windows/discovery/host/empire_find_localadmin_smb_svcctl_OpenSCManager.zip'
zipFileRequest = requests.get(url)
zipFile = ZipFile(BytesIO(zipFileRequest.content))
datasetJSONPath = zipFile.extract(zipFile.namelist()[0])

In [2]:
import pandas as pd
from pandas.io import json

df = json.read_json(path_or_buf=datasetJSONPath, lines=True)

C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\ly\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
(
df[['@timestamp','Hostname','SubjectUserName','ProcessName','ObjectName']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4656)
    & (df['ObjectType'] == 'SC_MANAGER OBJECT')
    & (df['ObjectName'] == 'ServicesActive')
    & (df['AccessMask'] == '0xf003f')
    & (df['SubjectLogonId'] != '0x3e4')
]
.head()
)

,@timestamp,Hostname,SubjectUserName,ProcessName,ObjectName
1071,2020-09-22T06:16:09.806Z,WORKSTATION6.theshire.local,pgustavo,C:\Windows\System32\services.exe,ServicesActive
1374,2020-09-22T06:16:30.920Z,MORDORDC.theshire.local,pgustavo,C:\Windows\System32\services.exe,ServicesActive


In [4]:
(
df[['@timestamp','Hostname','SubjectUserName','ProcessName','ObjectName','PrivilegeList','ObjectServer']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4674)
    & (df['ObjectType'] == 'SC_MANAGER OBJECT')
    & (df['ObjectName'] == 'ServicesActive')
    & (df['PrivilegeList'] == 'SeTakeOwnershipPrivilege')
    & (df['SubjectLogonId'] != '0x3e4')
]
.head()
)

,@timestamp,Hostname,SubjectUserName,ProcessName,ObjectName,PrivilegeList,ObjectServer
854,2020-09-22T06:16:08.637Z,WORKSTATION5.theshire.local,pgustavo,C:\Windows\System32\services.exe,ServicesActive,SeTakeOwnershipPrivilege,SC Manager


In [5]:
(
df[['@timestamp','Hostname','Application','SourcePort','SourceAddress','DestPort','DestAddress']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 5156)
    & (df['Application'].str.lower().str.endswith('services.exe', na=False))
    & (df['LayerRTID'] == 44)
]
)

,@timestamp,Hostname,Application,SourcePort,SourceAddress,DestPort,DestAddress
1066,2020-09-22T06:16:09.804Z,WORKSTATION6.theshire.local,\device\harddiskvolume2\windows\system32\servi...,54786.0,172.18.39.5,49726.0,172.18.39.6


In [6]:
(
df[['@timestamp','Hostname','User','SourcePort','SourceIp','DestinationPort','DestinationIp']]

[(df['Channel'] == 'Microsoft-Windows-Sysmon/Operational')
    & (df['EventID'] == 3)
    & (df['Image'].str.lower().str.endswith('services.exe', na=False))
]
)

,@timestamp,Hostname,User,SourcePort,SourceIp,DestinationPort,DestinationIp
1100,2020-09-22T06:16:11.808Z,WORKSTATION6.theshire.local,NT AUTHORITY\SYSTEM,54786.0,172.18.39.5,49726.0,172.18.39.6


In [15]:
handleRequestDf = (
# (
df[['@timestamp','Hostname','SubjectUserName','SubjectLogonId']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4656)
    & (df['SubjectLogonId'] != '0x3e4')
]
)

networkLogonDf = (
# (
df[['@timestamp', 'Hostname', 'TargetUserName', 'TargetLogonId', 'IpAddress']]

[(df['Channel'].str.lower() == 'security')
    & (df['EventID'] == 4624)
    & (df['LogonType'] == 3)
    & (~df['SubjectUserName'].str.lower().str.endswith('$', na=False))
]
)

(
pd.merge(handleRequestDf, networkLogonDf,
  left_on = 'SubjectLogonId', right_on = 'TargetLogonId', how = 'inner')
)

,@timestamp_x,Hostname_x,SubjectUserName,SubjectLogonId,@timestamp_y,Hostname_y,TargetUserName,TargetLogonId,IpAddress
0,2020-09-22T06:16:09.805Z,WORKSTATION6.theshire.local,pgustavo,0x463a69d,2020-09-22T06:16:09.804Z,WORKSTATION6.theshire.local,pgustavo,0x463a69d,172.18.39.5
1,2020-09-22T06:16:09.806Z,WORKSTATION6.theshire.local,pgustavo,0x463a69d,2020-09-22T06:16:09.804Z,WORKSTATION6.theshire.local,pgustavo,0x463a69d,172.18.39.5
2,2020-09-22T06:16:30.919Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,2020-09-22T06:16:30.918Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,172.18.39.5
3,2020-09-22T06:16:30.919Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,2020-09-22T06:16:30.918Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,172.18.39.5
4,2020-09-22T06:16:30.920Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,2020-09-22T06:16:30.918Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,172.18.39.5
5,2020-09-22T06:16:30.920Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,2020-09-22T06:16:30.918Z,MORDORDC.theshire.local,pgustavo,0x6fe8c94,172.18.39.5
